这段代码包括资源限制器修饰器resource_limiter，constrain_tools函数以及一个示例，演示如何使用资源限制器来限制AutoGPT的工具使用。在这个示例中，我们将搜索工具（serpapi）的资源消耗设置为5，llm-math工具的资源消耗设置为3。

In [3]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from functools import wraps

# 设置全局资源量
autogpt_resources = 50

# 资源限制器修饰器
def resource_limiter(resource_cost):
    global autogpt_resources

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            global autogpt_resources
            print(f"Autogpt resources: {autogpt_resources}")
            if autogpt_resources < resource_cost:
                raise ValueError(f"Not enough resources. Available: {autogpt_resources}, required: {resource_cost}")
            autogpt_resources -= resource_cost
            return func(*args, **kwargs)

        return wrapper

    return decorator

def constrain_tools(tool_costs, llm=None):
    tool_names = list(tool_costs.keys())
    resource_costs = list(tool_costs.values())
    tools = load_tools(tool_names, llm=llm)

    for i, tool in enumerate(tools):
        tool.func = resource_limiter(resource_costs[i])(tool.func)

    return tools

# 配置工具名称和对应的资源消耗
tool_costs = {"serpapi": 5, "llm-math": 3}

llm = ChatOpenAI(temperature=0)
tools = constrain_tools(tool_costs, llm=llm)
tools[0].name = "Google Search"

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")




> Entering new AgentExecutor chain...
I need to find out Leo DiCaprio's girlfriend's name and her age.
Action: Google Search
Action Input: "Leo DiCaprio girlfriend"Autogpt resources: 50
Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. The Titanic actor hasn't been in a relationship with a woman over the age of ...I need to find out Camila Morrone's age.
Action: Google Search
Action Input: "Camila Morrone age"Autogpt resources: 45
25 yearsI need to calculate her age raised to the 0.43 power.
Action: Calculator
Action Input: 25^(0.43)Autogpt resources: 40
Answer: 3.991298452658078I now know the final answer.
Final Answer: Camila Morrone's current age raised to the 0.43 power is approximately 3.99.

> Finished chain.


"Camila Morrone's current age raised to the 0.43 power is approximately 3.99."